# Feature Selection for Drug Overdose Data

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import patsy 
import math

pd.set_option('display.max_columns', 100)
%matplotlib inline

## Load CSV Files into Pandas DataFrames
The csv files are names as follows for each state:
* `{state}0.csv` - Introduction
* `{state}1.csv` - Outcomes & Factors Rankings
* `{state}2.csv` - Outcomes & Factors SubRankings
* `{state}3.csv` - Ranked Measure Data
* `{state}4.csv` - Additional Measure Data
* `{state}5.csv` - Ranked Measure Sources & Years
* `{state}6.csv` - Addtl Measure Sources & Years

The Additional Measure Data looks to contain the valuable information for this usecase.

In [2]:
states = ['alabama', 'florida', 'louisiana', 'nebraska', 'oklahoma', 'vermont', 'alaska', 
          'georgia', 'maine', 'nevada', 'oregon', 'virginia', 'arizona', 'hawaii', 
          'maryland', 'new-hampshire', 'pennsylvania', 'washington', 'arkansas', 'idaho',
          'massachusetts', 'new-jersey', 'rhode-island', 'west-virginia', 'california',
          'illinois', 'michigan', 'new-mexico', 'south-carolina', 'wisconsin', 'colorado',
          'indiana', 'minnesota', 'new-york', 'south-dakota', 'wyoming', 'connecticut', 
          'iowa', 'mississippi', 'north-carolina', 'tennessee', 'delaware', 'kansas',
          'missouri', 'north-dakota', 'texas', 'district-of-columbia', 'kentucky', 
          'montana', 'ohio', 'utah'
]

In [3]:
measure_df = pd.DataFrame()
for state in states:
    data_path = 'data/csv/{}.csv.4'.format(state)
    measure_df = pd.concat([measure_df, pd.read_csv(data_path)])

In [4]:
measure_df.describe()

,FIPS,# Deaths,Age-Adjusted Mortality,95% CI - Low,95% CI - High,Age-Adjusted Mortality (Black),Age-Adjusted Mortality (Hispanic),Age-Adjusted Mortality (White),# Deaths.1,Child Mortality Rate,95% CI - Low.1,95% CI - High.1,Child Mortality Rate (Black),Child Mortality Rate (Hispanic),Child Mortality Rate (White),# Deaths.2,Infant Mortality Rate,95% CI - Low.2,95% CI - High.2,Infant Mortality Rate (Black),Infant Mortality Rate (Hispanic),Infant Mortality Rate (White),% Frequent Physical Distress,95% CI - Low.3,95% CI - High.3,% Frequent Mental Distress,95% CI - Low.4,95% CI - High.4,% Diabetic,95% CI - Low.5,95% CI - High.5,# HIV Cases,HIV Prevalence Rate,# Food Insecure,% Food Insecure,# Limited Access,% Limited Access,# Drug Overdose Deaths,Drug Overdose Mortality Rate,Range Drug Overdose Mortality Rate,# Motor Vehicle Deaths,MV Mortality Rate,95% CI - Low.6,95% CI - High.6,% Insufficient Sleep,95% CI - Low.7,95% CI - High.7,# Uninsured,% Uninsured,95% CI - Low.8,95% CI - High.8,# Uninsured.1,% Uninsured.1,95% CI - Low.9,95% CI - High.9,Costs,Other PCP Rate,% Disconnected Youth,Household Income,95% CI - Low.10,95% CI - High.10,Household income (Black),Household income (Hispanic),Household income (White),% Free or Reduced Lunch,Segregation index,Segregation Index,Homicide Rate,95% CI - Low.11,95% CI - High.11,# Firearm Fatalities,Firearm Fatalities Rate,95% CI - Low.12,95% CI - High.12,Population,% < 18,% 65 and over,# African American,% African American,# American Indian/Alaskan Native,% American Indian/Alaskan Native,# Asian,% Asian,# Native Hawaiian/Other Pacific Islander,% Native Hawaiian/Other Pacific Islander,# Hispanic,% Hispanic,# Non-Hispanic White,% Non-Hispanic White,# Not Proficient in English,% Not Proficient in English,95% CI - Low.13,95% CI - High.13,% Female,# Rural,% Rural
count,3193.000000,3128.000000,3128.000000,3128.000000,3128.000000,1335.000000,798.000000,1557.000000,1982.000000,1982.000000,1982.000000,1982.000000,499.000000,360.000000,605.000000,1312.000000,1312.000000,1312.000000,1312.000000,350.000000,229.000000,405.000000,3193.000000,3193.000000,3193.000000,3193.000000,3193.000000,3193.000000,3193.000000,3143.000000,3143.000000,2475.000000,2475.000000,3.193000e+03,3193.000000,3.174000e+03,3174.000000,1706.000000,1706.000000,3185.000000,2742.000000,2742.000000,2742.000000,2742.000000,3193.000000,3193.000000,3193.000000,3.192000e+03,3192.000000,3192.000000,3192.000000,3192.000000,3192.000000,3192.000000,3192.000000,3186.000000,3158.000000,2119.000000,3192.000000,3192.000000,3192.000000,1939.000000,2458.000000,2759.000000,3173.000000,2102.000000,2837.000000,1296.000000,1296.000000,1296.000000,2236.000000,2236.000000,2236.000000,2236.000000,3.193000e+03,3193.000000,3193.000000,3.193000e+03,3193.000000,3193.000000,3193.000000,3.193000e+03,3193.000000,3193.000000,3193.000000,3.193000e+03,3193.000000,3.193000e+03,3193.000000,3.193000e+03,3193.000000,3017.000000,3017.000000,3193.000000,3.186000e+03,3186.000000
mean,30360.922643,2329.359974,400.472666,353.251982,451.146228,524.963446,254.895113,408.987861,147.260848,61.589908,41.042735,90.242028,96.495936,47.924560,49.389702,241.349848,6.874979,5.136738,9.071646,12.118114,5.168202,5.078608,11.983965,11.604198,12.373695,12.208911,11.867371,12.558964,11.350767,8.885746,14.254757,767.685657,190.820727,2.737386e+04,14.131475,1.148044e+04,8.615288,188.213951,19.835819,8.050557,177.711889,18.655892,13.041320,26.159117,33.057112,32.046344,34.082142,1.610335e+04,14.255740,12.406953,16.104527,2410.670426,6.534262,4.747028,8.321496,9624.307241,71.718584,18.526083,49660.226504,45138.036507,54182.416500,35625.556988,42240.774614,52496.852483,54.440246,45.782296,31.218844,6.078997,4.195833,8.694444,155.190966,14.645196,9.886047,21.217844,2.023974e+05,22.347379,18.342257,2.519839e+04,8.977250,2539.711243,2.303904,1.147418e+04,1.531183,483.228938,0.139056,3.599767e+04,9.328109,1.240023e+05,76.457899,8.393362e+03,1.773349,1.116786,2.682814,49.909942,3.733560e+04,58.058842
std

## Create Urban Feature


In [5]:
measure_df['URBAN'] = measure_df['% Rural'] <=50